In [1]:
api_key="pcsk_6bELL7_6GT9Pk4Eiz7JNEmRd6K3Gw2GnnyEVkvagPXnJTWyPs6fiXabdfeMughRknNntwk"

In [2]:
from langchain_community.retrievers.pinecone_hybrid_search import PineconeHybridSearchRetriever


In [3]:
import os

In [4]:
from pinecone import Pinecone,ServerlessSpec

In [5]:
index_name = "langchain-hybrid-search"

In [6]:
pc= Pinecone(
    api_key=api_key)

In [7]:
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="dotproduct",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

In [8]:
index=pc.Index(index_name)
index

c:\Users\mithlesh singla\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import os

from dotenv import load_dotenv
load_dotenv()


True

In [10]:
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings 
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
from pinecone_text.sparse import BM25Encoder
bm_25 = BM25Encoder().default()

[nltk_data] Downloading package punkt_tab to C:\Users\mithlesh
[nltk_data]     singla\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package stopwords to C:\Users\mithlesh
[nltk_data]     singla\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [15]:
from pinecone_text.sparse import BM25Encoder

sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "A journey of a thousand miles begins with a single step.",
    "To be or not to be, that is the question."
]

# Fit and dump
bm_25 = BM25Encoder()
bm_25.fit(sentences)
bm_25.dump("bm25_values.json")

# Load using a new instance
bm_25_loaded = BM25Encoder()
bm_25_loaded.load("bm25_values.json")


100%|██████████| 3/3 [00:00<00:00, 1426.15it/s]


In [16]:
retriver= PineconeHybridSearchRetriever(embeddings=embeddings,
    sparse_encoder=bm_25_loaded,index=index)

In [17]:
retriver.add_texts(sentences)

100%|██████████| 1/1 [00:03<00:00,  3.51s/it]


In [19]:
retriver.invoke("What is the colour of fox?")

[Document(metadata={'score': 0.35122788}, page_content='The quick brown fox jumps over the lazy dog.'),
 Document(metadata={'score': 0.043923378}, page_content='To be or not to be, that is the question.'),
 Document(metadata={'score': 0.00463056564}, page_content='A journey of a thousand miles begins with a single step.')]